The objective of this notebook is to choose a model for predicting if it is hazardous. To be complete it has to achieve these objectives.
- the model has to have the highest F1 score because in the context of predicting a potentially hazardous asteroid a false negative would be very deadly, and a false positive would not be ideal
- We will use RandomizedSearchCV because it is fast and randomized without being exhaustive 

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from src.pipeline import build_pipeline

Next We will load the data and process using the pipeline.

In [12]:
data = pd.read_csv('../data/raw/asteroids_data.csv')

/var/folders/20/c46fqz652sjfpnm1mf8ssp5m0000gn/T/ipykernel_62238/538008766.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/raw/asteroids_data.csv')


In [ ]:
pipeline = build_pipeline()
X = data.drop(columns=['is_potentially_hazardous_asteroid'])
y = data['is_potentially_hazardous_asteroid']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)